In [4]:
from ollama import chat
from pydantic import BaseModel

import json
from langchain_ollama import ChatOllama

# Set up the local Ollama LLM with llama3.2
llm = ChatOllama(model="llama3.2")


class LLMResponse(BaseModel):
  # uuid: list[str]
  question: str
  extracted_content: str
  answer: str


with open('conversations.json') as json_data:
  data = json.load(json_data)
  # print(data)

def getLLMResponse(msgObj):
  response = chat(
    messages=[
      {
        'role': 'user',
        'content': 'You have been given a JSON object that contains a conversation between an assistant and a user. Extract and return only the data related to ReasonReact. Review all chat messages, extracted content, and assistant responses to determine the question being asked (with code) and the assistant response (with code).' + str(msgObj),
      }
    ],
    model='llama3.2',
    format=LLMResponse.model_json_schema(),
  )

  answer = response.message.content
  responseJson = json.loads(answer)
  # if answer.question and if answer.answer, then append to convo.jsonl
  # print(responseJson)
  # if responseJson.get('question') and responseJson.get('answer'):
  #   # append answer to convo.jsonl
  #   with open('conversations.jsonl', 'a') as f:
  #       f.write(answer)
  #       f.write('\n')
  #       f.close()

  inputs = {
    "question": responseJson.get('question'),
    "associatedData": responseJson.get('extracted_content')
  }

  outputs = {
    "output": responseJson.get('answer')
  }

  resObj = {
    "inputs": inputs,
    "outputs": outputs
  }
  # criteria = False;
  # if any value is null, then restart the process
  # while criteria == False:
  if not resObj['inputs']['question'] or not resObj['outputs']['output']:
    # append answer to convo.jsonl
    print(resObj)
  else:
    print(resObj)
    qualNumber = qualityCheck(resObj)
    print(qualNumber)
    if qualNumber > 3:
      # criteria = True
      with open('conversations_2.jsonl', 'a') as f:
          f.write(json.dumps(resObj))
          f.write('\n')
          f.close()
    
class QualityCheckResponse(BaseModel):
  qualityRank: int
  reasoning: str

def qualityCheck(msgObj: str):
  response = chat(
    messages=[
      {
        'role': 'user',
        'content': 'You have been given a training data that contains a conversation between an assistant and a user. Assign a quality ranking 0 to 5 with 5 being the best to determine if the data is properly formatted and makes sense. The goal is to create quality training data. Give your reasoning as well. here is the content to check:' + str(msgObj),
      }
    ],
    model='llama3.2',
    format=QualityCheckResponse.model_json_schema(),
  )

  answer = response.message.content
  responseJson = json.loads(answer)
  # if answer.question and if answer.answer, then append to convo.jsonl
  print(responseJson)
  return int(responseJson.get('qualityRank'))

def runForAllData(data):
  for i in range(len(data) - 1):
    getLLMResponse(data[i])

# print(data[0])
for _ in range(100):
  runForAllData(data);

{'inputs': {'question': 'What is the issue with the test suite?', 'associatedData': 'Call retries were exceeded'}, 'outputs': {'output': 'The test suite failed to run due to exceeding call retries.'}}
{'qualityRank': 5, 'reasoning': "The data appears to be properly formatted, as it contains the expected structure of a conversational dialogue. The input and output are clearly defined, making it easy to understand the user's question and the assistant's response. Additionally, the associated data provides context to the conversation, which is helpful for training AI models. Overall, this data seems to be well-structured and follows best practices for creating quality training data."}
5
{'inputs': {'question': 'Write the simplest test for Tools component', 'associatedData': 'Tools with Some empty list\n2. Tools with Some non-empty list\n3. Different tool configurations\n'}, 'outputs': {'output': 'Tools with None input'}}
{'qualityRank': 4, 'reasoning': "The provided training data seems to

KeyboardInterrupt: 